# Complete Risk Model Pipeline - Full Demo
## Tek Pipeline ile Tüm İşlemler

Bu notebook, tüm risk modelleme sürecini tek bir pipeline ile gerçekleştirir:

### İçerik:
1. **Veri Oluşturma** - Gerçekçi kredi risk verisi (60-70% Gini)
2. **Pipeline Kurulumu** - Config ile tüm kontrol
3. **WOE & Univariate Gini** - Her değişken için
4. **Feature Selection** - PSI → VIF → Correlation → IV → Boruta → Stepwise
5. **Model Training** - Logistic, GAM, CatBoost, LightGBM, XGBoost, RF, ExtraTrees
6. **Two-Stage Calibration** - Stage 1 (long-run) + Stage 2 (recent)
7. **Risk Bands** - Optimizasyon ve testler
8. **Scoring** - Yeni veri skorlama
9. **Reporting** - Tüm sonuçlar ve grafikler

### Özellikler:
- ✓ Tek pipeline (UnifiedRiskPipeline)
- ✓ Config ile tam kontrol
- ✓ Scoring default olarak kapalı
- ✓ Data dictionary desteği
- ✓ Two-stage calibration
- ✓ Forward/Backward/Stepwise selection
- ✓ Boruta with LightGBM
- ✓ WOE optimizasyonu (IV/Gini)
- ✓ Noise sentinel kontrolü
- ✓ Equal default rate splits
- ✓ Risk band optimization
- ✓ Tüm ML algoritmaları

## 1. Setup & İmportlar

In [ ]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add source to path
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report

# Pipeline imports
from risk_pipeline.unified_pipeline import UnifiedRiskPipeline
from risk_pipeline.core.config import Config

# Visualization settings
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
sns.set_style('whitegrid')
sns.set_palette('husl')

# FIXED SEED
SEED = 42
np.random.seed(SEED)

print("✓ Setup complete")
print(f"✓ Random seed: {SEED}")
print(f"✓ Working directory: {os.getcwd()}")

## 2. Veri Oluşturma - Gerçekçi Kredi Risk Verisi

In [ ]:
def create_realistic_credit_data(n_samples=20000, seed=42):
    """
    Gerçekçi kredi risk verisi oluştur.
    Hedef: 60-70% Gini, en az 5 değişken seçilecek
    """
    np.random.seed(seed)
    
    # Zaman serisi (OOT split için)
    start_date = datetime(2022, 1, 1)
    dates = pd.date_range(start=start_date, periods=n_samples, freq='H')[:n_samples]
    
    df = pd.DataFrame({
        'app_id': [f'APP_{i:06d}' for i in range(n_samples)],
        'app_dt': dates
    })
    
    # NUMERİK DEĞİŞKENLER (10 adet)
    
    # 1. Kredi Skoru (300-850) - GÜÇLÜ
    df['credit_score'] = np.random.beta(5, 2, n_samples) * 550 + 300
    df['credit_score'] = np.clip(df['credit_score'], 300, 850).round()
    
    # 2. Yıllık Gelir - GÜÇLÜ
    df['annual_income'] = np.random.lognormal(10.8, 0.6, n_samples)
    df['annual_income'] = np.clip(df['annual_income'], 20000, 500000).round()
    
    # 3. Borç/Gelir Oranı - GÜÇLÜ
    df['debt_to_income_ratio'] = np.random.beta(2, 5, n_samples) * 100
    
    # 4. İstihdam Yılı - ORTA
    df['employment_length'] = np.random.gamma(2, 3, n_samples)
    df['employment_length'] = np.clip(df['employment_length'], 0, 40).round()
    
    # 5. Kredi Sayısı - ORTA
    df['number_of_credit_lines'] = np.random.negative_binomial(5, 0.5, n_samples)
    df['number_of_credit_lines'] = np.clip(df['number_of_credit_lines'], 0, 30)
    
    # 6. Kredi Tutarı - ORTA
    df['loan_amount'] = np.random.lognormal(9.2, 0.7, n_samples)
    df['loan_amount'] = np.clip(df['loan_amount'], 1000, 100000).round()
    
    # 7. Son Gecikme (aylar) - GÜÇLÜ (%60 missing)
    df['months_since_last_delinquency'] = np.where(
        np.random.rand(n_samples) < 0.6,
        np.nan,
        np.random.exponential(24, n_samples)
    )
    
    # 8. Kredi Kullanım Oranı - GÜÇLÜ
    df['credit_utilization_rate'] = np.random.beta(2, 3, n_samples) * 100
    
    # 9. Son 6 Ayda Sorgu - ORTA
    df['inquiries_last_6_months'] = np.random.poisson(0.5, n_samples)
    df['inquiries_last_6_months'] = np.clip(df['inquiries_last_6_months'], 0, 10)
    
    # 10. Yaş - ZAYIF
    df['age'] = np.random.normal(42, 12, n_samples)
    df['age'] = np.clip(df['age'], 18, 75).round().astype(int)
    
    # KATEGORİK DEĞİŞKENLER (6 adet)
    
    # 1. Ev Sahipliği - GÜÇLÜ
    df['home_ownership'] = np.random.choice(
        ['RENT', 'OWN', 'MORTGAGE', 'OTHER'],
        n_samples, p=[0.35, 0.15, 0.45, 0.05]
    )
    
    # 2. Kredi Amacı - ORTA
    df['loan_purpose'] = np.random.choice(
        ['debt_consolidation', 'credit_card', 'home_improvement', 
         'major_purchase', 'small_business', 'other'],
        n_samples, p=[0.35, 0.20, 0.15, 0.10, 0.10, 0.10]
    )
    
    # 3. İstihdam Tipi - ORTA
    df['employment_type'] = np.random.choice(
        ['Full-time', 'Part-time', 'Self-employed', 'Retired', 'Student', 'Unemployed'],
        n_samples, p=[0.55, 0.10, 0.15, 0.10, 0.05, 0.05]
    )
    
    # 4. Eğitim Seviyesi - ZAYIF
    df['education_level'] = np.random.choice(
        ['High School', 'Associate', 'Bachelor', 'Master', 'Doctorate'],
        n_samples, p=[0.25, 0.20, 0.35, 0.15, 0.05]
    )
    
    # 5. Medeni Durum - ZAYIF
    df['marital_status'] = np.random.choice(
        ['Single', 'Married', 'Divorced', 'Widowed'],
        n_samples, p=[0.30, 0.50, 0.15, 0.05]
    )
    
    # 6. Eyalet - ZAYIF
    df['state'] = np.random.choice(
        ['CA', 'NY', 'TX', 'FL', 'IL', 'PA', 'OH', 'GA', 'NC', 'MI', 'Other'],
        n_samples, p=[0.12, 0.10, 0.10, 0.08, 0.07, 0.07, 0.06, 0.06, 0.06, 0.06, 0.22]
    )
    
    # HEDEF DEĞİŞKEN - Güçlü ilişkilerle
    risk_score = (
        # Kredi skoru etkisi (normalize)
        -2.0 * (df['credit_score'] - 300) / 550 +
        
        # Gelir etkisi (log scale)
        -0.5 * np.log(df['annual_income'] / 50000) +
        
        # DTI etkisi
        0.02 * df['debt_to_income_ratio'] +
        
        # Kullanım oranı etkisi
        0.01 * df['credit_utilization_rate'] +
        
        # İstihdam yılı etkisi
        -0.03 * np.sqrt(df['employment_length']) +
        
        # Sorgu etkisi
        0.2 * df['inquiries_last_6_months'] +
        
        # Kredi sayısı etkisi
        -0.02 * np.log(df['number_of_credit_lines'] + 1) +
        
        # Kategorik etkiler
        np.where(df['home_ownership'] == 'RENT', 0.3, 0) +
        np.where(df['home_ownership'] == 'OWN', -0.2, 0) +
        np.where(df['employment_type'] == 'Unemployed', 0.5, 0) +
        np.where(df['employment_type'] == 'Full-time', -0.1, 0) +
        np.where(df['loan_purpose'] == 'debt_consolidation', 0.2, 0) +
        
        # Gecikme etkisi (çok güçlü)
        np.where(df['months_since_last_delinquency'].isna(), -0.2,
                np.where(df['months_since_last_delinquency'] < 12, 0.6,
                        np.where(df['months_since_last_delinquency'] < 24, 0.3, 0.1))) +
        
        # Kontrollü gürültü
        np.random.randn(n_samples) * 0.3
    )
    
    # Binary hedef oluştur
    prob = 1 / (1 + np.exp(-risk_score))
    df['target'] = ((prob + np.random.rand(n_samples) * 0.15) > 0.58).astype(int)
    
    # Default rate'i ayarla (~%12-15)
    if df['target'].mean() > 0.15:
        threshold = np.percentile(risk_score, 85)
        df['target'] = (risk_score > threshold).astype(int)
    
    return df

# Veri oluştur
print("Veri oluşturuluyor...")
df_main = create_realistic_credit_data(n_samples=20000, seed=SEED)

print(f"✓ Veri oluşturuldu: {len(df_main):,} kayıt")
print(f"✓ Değişkenler: {len(df_main.columns)} ({len(df_main.select_dtypes(include=[np.number]).columns)} numerik, {len(df_main.select_dtypes(include=['object']).columns)} kategorik)")
print(f"✓ Default rate: {df_main['target'].mean():.2%}")
print(f"✓ Tarih aralığı: {df_main['app_dt'].min().date()} - {df_main['app_dt'].max().date()}")

## 3. Veri Sözlüğü Oluşturma (Opsiyonel)

In [ ]:
# Data Dictionary - Değişken açıklamaları
data_dictionary = pd.DataFrame([
    {'variable': 'credit_score', 'description': 'FICO kredi skoru (300-850)', 'type': 'numeric', 'category': 'Credit'},
    {'variable': 'annual_income', 'description': 'Yıllık gelir (USD)', 'type': 'numeric', 'category': 'Financial'},
    {'variable': 'debt_to_income_ratio', 'description': 'Toplam borç/gelir oranı (%)', 'type': 'numeric', 'category': 'Financial'},
    {'variable': 'employment_length', 'description': 'İstihdam süresi (yıl)', 'type': 'numeric', 'category': 'Employment'},
    {'variable': 'number_of_credit_lines', 'description': 'Açık kredi sayısı', 'type': 'numeric', 'category': 'Credit'},
    {'variable': 'loan_amount', 'description': 'Talep edilen kredi tutarı', 'type': 'numeric', 'category': 'Loan'},
    {'variable': 'months_since_last_delinquency', 'description': 'Son gecikmeden bu yana geçen ay', 'type': 'numeric', 'category': 'Credit'},
    {'variable': 'credit_utilization_rate', 'description': 'Kredi kullanım oranı (%)', 'type': 'numeric', 'category': 'Credit'},
    {'variable': 'inquiries_last_6_months', 'description': 'Son 6 ayda kredi sorgusu', 'type': 'numeric', 'category': 'Credit'},
    {'variable': 'age', 'description': 'Müşteri yaşı', 'type': 'numeric', 'category': 'Demographic'},
    {'variable': 'home_ownership', 'description': 'Ev sahipliği durumu', 'type': 'categorical', 'category': 'Demographic'},
    {'variable': 'loan_purpose', 'description': 'Kredi kullanım amacı', 'type': 'categorical', 'category': 'Loan'},
    {'variable': 'employment_type', 'description': 'İstihdam tipi', 'type': 'categorical', 'category': 'Employment'},
    {'variable': 'education_level', 'description': 'Eğitim seviyesi', 'type': 'categorical', 'category': 'Demographic'},
    {'variable': 'marital_status', 'description': 'Medeni durum', 'type': 'categorical', 'category': 'Demographic'},
    {'variable': 'state', 'description': 'İkamet edilen eyalet', 'type': 'categorical', 'category': 'Geographic'}
])

print("Veri Sözlüğü:")
print(data_dictionary[['variable', 'description', 'category']].to_string(index=False))

## 4. Pipeline Konfigürasyonu

In [ ]:
# KAPSAMLI KONFİGÜRASYON
config = Config(
    # === TEMEL AYARLAR ===
    target_col='target',
    id_col='app_id',
    time_col='app_dt',
    random_state=SEED,
    
    # === FEATURE SELECTION AYARLARI ===
    # Thresholds
    iv_min=0.02,  # Minimum IV değeri
    iv_threshold=0.02,  # IV threshold for selection
    iv_high_threshold=2.0,  # Maksimum IV (leakage kontrolü)
    psi_threshold=0.25,  # PSI threshold
    rho_threshold=0.85,  # Correlation threshold
    correlation_threshold=0.85,  # Alternatif isim
    vif_threshold=10.0,  # VIF threshold
    rare_threshold=0.01,  # Nadir kategori threshold
    
    # Selection sırası ve yöntemi
    selection_order=['psi', 'vif', 'correlation', 'iv', 'boruta', 'stepwise'],
    selection_method='stepwise',  # 'forward', 'backward', 'stepwise'
    max_features=20,
    min_features=5,
    
    # === WOE AYARLARI ===
    enable_woe=True,
    binning_method='optimized',  # 'quantile', 'equal_width', 'optimized'
    n_bins=10,  # Başlangıç bin sayısı
    max_bins=20,  # Maksimum bin sayısı
    min_bin_size=0.05,  # Minimum bin büyüklüğü
    woe_monotonic=True,  # Monoton WOE zorla
    
    # === MODEL AYARLARI ===
    model_type='all',  # 'logistic', 'lgbm', 'xgboost', 'gam', 'catboost', 'rf', 'extratrees', 'all'
    use_optuna=False,  # Hyperparameter optimization
    n_trials=50,  # Optuna trial sayısı
    cv_folds=5,  # Cross-validation fold sayısı
    
    # === PIPELINE AYARLARI ===
    enable_dual_pipeline=True,  # Hem WOE hem RAW modeller
    enable_noise_sentinel=True,  # Noise değişken kontrolü
    enable_calibration=True,  # Kalibrasyon
    enable_scoring=False,  # Scoring (default kapalı)
    
    # === SPLIT AYARLARI ===
    test_ratio=0.2,  # Test set oranı
    oot_months=3,  # OOT için son kaç ay
    equal_default_splits=True,  # Equal default rate in splits
    min_oot_size=1000,  # Minimum OOT size
    
    # === CALIBRATION AYARLARI ===
    calibration_method='isotonic',  # 'isotonic', 'sigmoid', 'platt', 'beta'
    stage2_method='lower_mean',  # 'lower_mean', 'upper_bound', 'weighted'
    
    # === RISK BAND AYARLARI ===
    n_risk_bands=10,  # Risk band sayısı
    band_method='quantile',  # 'quantile', 'equal_width', 'optimal'
    
    # === OUTPUT AYARLARI ===
    output_folder='../output_demo',
    save_plots=True,
    save_model=True,
    save_reports=True
)

print("Pipeline Konfigürasyonu:")
print(f"  WOE: {config.enable_woe}")
print(f"  Dual Pipeline: {config.enable_dual_pipeline}")
print(f"  Noise Sentinel: {config.enable_noise_sentinel}")
print(f"  Calibration: {config.enable_calibration}")
print(f"  Scoring: {config.enable_scoring}")
print(f"  Selection Order: {' → '.join(config.selection_order)}")
print(f"  Selection Method: {config.selection_method}")
print(f"  Model Type: {config.model_type}")

## 5. Pipeline Çalıştırma - ADIM ADIM

In [ ]:
# Pipeline oluştur
print("Pipeline oluşturuluyor...\n")
pipeline = UnifiedRiskPipeline(config)

print("="*80)
print("PIPELINE ÇALIŞTIRILIYOR")
print("="*80)

In [ ]:
# Kalibrasyon verisi (opsiyonel)
# Long-run average için tüm veriyi kullan
calibration_df = df_main.copy()

# Stage 2 için son 3 aylık veri
stage2_cutoff = df_main['app_dt'].max() - pd.Timedelta(days=90)
stage2_df = df_main[df_main['app_dt'] >= stage2_cutoff].copy()

print(f"Calibration data: {len(calibration_df):,} kayıt")
print(f"Stage 2 data: {len(stage2_df):,} kayıt (son 3 ay)")

In [ ]:
# PIPELINE FIT
results = pipeline.fit(
    df=df_main,
    data_dictionary=data_dictionary,  # Opsiyonel
    calibration_df=calibration_df,    # Opsiyonel - Stage 1 için
    stage2_df=stage2_df,              # Opsiyonel - Stage 2 için
    score_df=None                      # Şimdilik None, sonra ekleyeceğiz
)

print("\n" + "="*80)
print("✓ Pipeline başarıyla tamamlandı!")
print("="*80)

## 6. Sonuçları İnceleme

In [ ]:
# SEÇILEN DEĞIŞKENLER
if 'selected_features' in results:
    print(f"SEÇİLEN DEĞİŞKENLER: {len(results['selected_features'])}")
    print("="*50)
    for i, feat in enumerate(results['selected_features'], 1):
        print(f"  {i:2d}. {feat}")
    
    if len(results['selected_features']) >= 5:
        print(f"\n✓ En az 5 değişken seçildi: {len(results['selected_features'])} değişken")
    else:
        print(f"\n✗ 5'ten az değişken seçildi: {len(results['selected_features'])} değişken")

In [ ]:
# WOE & UNIVARIATE GINI
if 'woe_results' in results:
    print("\nWOE & UNIVARIATE GINI ANALİZİ")
    print("="*80)
    
    woe_results = results['woe_results']
    
    # Univariate Gini tablosu
    if 'univariate_gini' in woe_results:
        gini_data = []
        for feat, gini_info in woe_results['univariate_gini'].items():
            gini_data.append({
                'Feature': feat,
                'Gini_Raw': gini_info.get('gini_raw', 0),
                'Gini_WOE': gini_info.get('gini_woe', 0),
                'Gini_Drop': gini_info.get('gini_drop', 0),
                'WOE_Quality': 'Good' if gini_info.get('gini_drop', 0) < 0.2 else 'Check'
            })
        
        gini_df = pd.DataFrame(gini_data)
        gini_df = gini_df.sort_values('Gini_WOE', ascending=False)
        
        print("Top 10 değişken (Univariate Gini):")
        print(gini_df.head(10).to_string(index=False, float_format='%.3f'))
    
    # IV değerleri
    if 'woe_values' in woe_results:
        print("\nInformation Value (IV) Analizi:")
        iv_data = []
        for feat, woe_info in woe_results['woe_values'].items():
            if isinstance(woe_info, dict) and 'iv' in woe_info:
                iv_data.append({'Feature': feat, 'IV': woe_info['iv']})
        
        if iv_data:
            iv_df = pd.DataFrame(iv_data).sort_values('IV', ascending=False)
            print(iv_df.head(10).to_string(index=False, float_format='%.4f'))

In [ ]:
# MODEL PERFORMANSLARI
if 'scores' in results:
    print("\nMODEL PERFORMANSLARI")
    print("="*80)
    
    perf_data = []
    for model_name, scores in results['scores'].items():
        row = {'Model': model_name}
        
        # Train performans
        if 'train_auc' in scores:
            row['Train_AUC'] = scores['train_auc']
            row['Train_Gini'] = (scores['train_auc'] * 2 - 1)
        
        # Test performans
        if 'test_auc' in scores:
            row['Test_AUC'] = scores['test_auc']
            row['Test_Gini'] = (scores['test_auc'] * 2 - 1)
        
        # OOT performans
        if 'oot_auc' in scores:
            row['OOT_AUC'] = scores['oot_auc']
            row['OOT_Gini'] = (scores['oot_auc'] * 2 - 1)
        
        perf_data.append(row)
    
    perf_df = pd.DataFrame(perf_data)
    
    # Gini değerlerini yüzdeye çevir
    for col in perf_df.columns:
        if 'Gini' in col:
            perf_df[col] = perf_df[col] * 100
    
    # Sırala ve göster
    if 'Test_Gini' in perf_df.columns:
        perf_df = perf_df.sort_values('Test_Gini', ascending=False)
    
    print(perf_df.to_string(index=False, float_format='%.2f'))
    
    # En iyi model
    if 'best_model_name' in results:
        print(f"\n✓ EN İYİ MODEL: {results['best_model_name']}")
        
        # Gini kontrolü
        best_scores = results['scores'].get(results['best_model_name'], {})
        if 'test_auc' in best_scores:
            test_gini = (best_scores['test_auc'] * 2 - 1) * 100
            if 60 <= test_gini <= 70:
                print(f"✓ Test Gini {test_gini:.1f}% - Hedef aralıkta (60-70%)")
            else:
                print(f"✗ Test Gini {test_gini:.1f}% - Hedef dışında")

In [ ]:
# CALIBRATION SONUÇLARI
if 'calibration_stage1' in results:
    print("\nCALIBRATION SONUÇLARI")
    print("="*50)
    
    # Stage 1
    stage1 = results['calibration_stage1']
    if 'calibration_metrics' in stage1:
        metrics = stage1['calibration_metrics']
        print("Stage 1 Calibration (Long-run Average):")
        print(f"  ECE (Expected Calibration Error): {metrics.get('ece', 0):.4f}")
        print(f"  MCE (Maximum Calibration Error): {metrics.get('mce', 0):.4f}")
        print(f"  Brier Score: {metrics.get('brier_score', 0):.4f}")
    
    # Stage 2
    if 'calibration_stage2' in results:
        stage2 = results['calibration_stage2']
        if 'adjustment_factor' in stage2:
            print(f"\nStage 2 Calibration (Recent Period):")
            print(f"  Method: {config.stage2_method}")
            print(f"  Adjustment Factor: {stage2['adjustment_factor']:.3f}")

In [ ]:
# RISK BANDS
if 'risk_bands' in results and results['risk_bands']:
    print("\nRISK BANDS ANALİZİ")
    print("="*50)
    
    bands = results['risk_bands']
    
    if 'bands' in bands:
        band_df = bands['bands']
        print("Risk Band Dağılımı:")
        print(band_df[['band', 'count', 'bad_rate', 'avg_score']].to_string(index=False))
    
    if 'metrics' in bands:
        metrics = bands['metrics']
        print(f"\nRisk Band Metrikleri:")
        print(f"  Herfindahl Index: {metrics.get('herfindahl_index', 0):.4f}")
        print(f"  Entropy: {metrics.get('entropy', 0):.4f}")
        print(f"  Gini Coefficient: {metrics.get('gini_coefficient', 0):.4f}")
        print(f"  KS Statistic: {metrics.get('ks_stat', 0):.4f}")
        print(f"  Hosmer-Lemeshow p-value: {metrics.get('hosmer_lemeshow_p', 0):.4f}")
        
        # Binomial test sonuçları
        if 'binomial_tests' in metrics:
            print("\nBinomial Test Sonuçları:")
            for test in metrics['binomial_tests'][:5]:  # İlk 5 band
                print(f"  Band {test['band']}: p-value = {test['p_value']:.4f}")

## 7. Model Görselleştirme

In [ ]:
# ROC Curves
if 'best_model' in results and results['best_model'] is not None:
    
    # Test verisi al
    test_data = pipeline.test_ if hasattr(pipeline, 'test_') else None
    oot_data = pipeline.oot_ if hasattr(pipeline, 'oot_') else None
    
    if test_data is not None:
        selected_features = results.get('selected_features', [])
        
        if selected_features:
            # Test predictions
            X_test = test_data[selected_features].fillna(0)
            y_test = test_data[config.target_col]
            test_pred = results['best_model'].predict_proba(X_test)[:, 1]
            
            # ROC hesapla
            test_fpr, test_tpr, _ = roc_curve(y_test, test_pred)
            test_auc = roc_auc_score(y_test, test_pred)
            test_gini = (test_auc * 2 - 1)
            
            # OOT predictions (varsa)
            oot_fpr, oot_tpr, oot_auc, oot_gini = None, None, None, None
            if oot_data is not None and len(oot_data) > 0:
                X_oot = oot_data[selected_features].fillna(0)
                y_oot = oot_data[config.target_col]
                oot_pred = results['best_model'].predict_proba(X_oot)[:, 1]
                oot_fpr, oot_tpr, _ = roc_curve(y_oot, oot_pred)
                oot_auc = roc_auc_score(y_oot, oot_pred)
                oot_gini = (oot_auc * 2 - 1)
            
            # Plot
            plt.figure(figsize=(10, 6))
            
            # Test ROC
            plt.plot(test_fpr, test_tpr, 'b-', lw=2, 
                    label=f'Test (AUC={test_auc:.3f}, Gini={test_gini:.1%})')
            
            # OOT ROC
            if oot_fpr is not None:
                plt.plot(oot_fpr, oot_tpr, 'r-', lw=2,
                        label=f'OOT (AUC={oot_auc:.3f}, Gini={oot_gini:.1%})')
            
            # Random line
            plt.plot([0, 1], [0, 1], 'k--', lw=1, alpha=0.5)
            
            plt.xlabel('False Positive Rate', fontsize=12)
            plt.ylabel('True Positive Rate', fontsize=12)
            plt.title(f'ROC Curve - {results["best_model_name"]}', fontsize=14)
            plt.legend(loc='lower right')
            plt.grid(alpha=0.3)
            plt.tight_layout()
            plt.show()
            
            # Performans özeti
            print("Model Performans Özeti:")
            print(f"  Test - AUC: {test_auc:.3f}, Gini: {test_gini:.1%}")
            if oot_auc:
                print(f"  OOT  - AUC: {oot_auc:.3f}, Gini: {oot_gini:.1%}")

## 8. Scoring - Yeni Veri Skorlama

In [ ]:
# Yeni veri oluştur (scoring için)
print("Scoring için yeni veri oluşturuluyor...")
df_score = create_realistic_credit_data(n_samples=1000, seed=SEED+1)

print(f"✓ Score verisi: {len(df_score):,} kayıt")
print(f"✓ Default rate: {df_score['target'].mean():.2%}")

In [ ]:
# SCORING YAP
if 'best_model' in results and results['best_model'] is not None:
    print("Scoring yapılıyor...\n")
    
    # Predict metodunu kullan
    scores = pipeline.predict(df_score)
    
    # Sonuçları kontrol et
    print("Scoring Sonuçları:")
    print(f"  Skorlanan kayıt: {len(scores):,}")
    print(f"  Ortalama skor: {scores['score'].mean():.3f}")
    print(f"  Median skor: {scores['score'].median():.3f}")
    print(f"  Min skor: {scores['score'].min():.3f}")
    print(f"  Max skor: {scores['score'].max():.3f}")
    
    # Performans kontrolü
    if 'target' in df_score.columns:
        score_auc = roc_auc_score(df_score['target'], scores['score'])
        score_gini = (score_auc * 2 - 1)
        print(f"\nScoring Performansı:")
        print(f"  AUC: {score_auc:.3f}")
        print(f"  Gini: {score_gini:.1%}")
    
    # Skor dağılımı
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.hist(scores['score'], bins=30, edgecolor='black', alpha=0.7)
    plt.xlabel('Score')
    plt.ylabel('Frequency')
    plt.title('Score Distribution')
    plt.grid(alpha=0.3)
    
    plt.subplot(1, 2, 2)
    if 'target' in df_score.columns:
        for target in [0, 1]:
            mask = df_score['target'] == target
            plt.hist(scores.loc[mask, 'score'], bins=30, alpha=0.5,
                    label=f'Target={target}')
        plt.xlabel('Score')
        plt.ylabel('Frequency')
        plt.title('Score Distribution by Target')
        plt.legend()
        plt.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Model bulunamadı, scoring yapılamıyor.")

## 9. Tek Seferde Çalıştırma (End-to-End)

In [ ]:
# TEK SEFERDE TÜM PIPELINE
def run_complete_pipeline():
    """
    Tüm pipeline'ı tek seferde çalıştır
    """
    print("="*80)
    print("COMPLETE PIPELINE - TEK SEFERDE ÇALIŞTIRMA")
    print("="*80)
    
    # 1. Veri oluştur
    print("\n[1/5] Veri oluşturuluyor...")
    df = create_realistic_credit_data(n_samples=15000, seed=SEED)
    print(f"✓ {len(df):,} kayıt oluşturuldu")
    
    # 2. Konfigürasyon
    print("\n[2/5] Pipeline konfigüre ediliyor...")
    config = Config(
        target_col='target',
        id_col='app_id',
        time_col='app_dt',
        random_state=SEED,
        
        # Ana ayarlar
        enable_woe=True,
        enable_dual_pipeline=True,
        enable_calibration=True,
        enable_scoring=False,
        
        # Selection
        selection_order=['psi', 'vif', 'correlation', 'iv', 'stepwise'],
        selection_method='forward',
        
        # Models
        model_type='all',
        
        # Output
        output_folder='../output_complete',
        save_model=True
    )
    print("✓ Konfigürasyon hazır")
    
    # 3. Pipeline oluştur ve fit et
    print("\n[3/5] Pipeline çalıştırılıyor...")
    pipeline = UnifiedRiskPipeline(config)
    
    # Kalibrasyon verileri
    calibration_df = df.copy()
    stage2_cutoff = df['app_dt'].max() - pd.Timedelta(days=90)
    stage2_df = df[df['app_dt'] >= stage2_cutoff].copy()
    
    results = pipeline.fit(
        df=df,
        calibration_df=calibration_df,
        stage2_df=stage2_df
    )
    print("✓ Pipeline tamamlandı")
    
    # 4. Sonuçları özetle
    print("\n[4/5] Sonuçlar:")
    if 'selected_features' in results:
        print(f"  • Seçilen değişkenler: {len(results['selected_features'])}")
    
    if 'best_model_name' in results:
        print(f"  • En iyi model: {results['best_model_name']}")
        
        if results['best_model_name'] in results.get('scores', {}):
            scores = results['scores'][results['best_model_name']]
            if 'test_auc' in scores:
                test_gini = (scores['test_auc'] * 2 - 1) * 100
                print(f"  • Test Gini: {test_gini:.1f}%")
    
    # 5. Scoring
    print("\n[5/5] Scoring test...")
    score_df = create_realistic_credit_data(n_samples=500, seed=SEED+10)
    scores = pipeline.predict(score_df)
    print(f"✓ {len(scores)} kayıt skorlandı")
    
    print("\n" + "="*80)
    print("✓ TÜM İŞLEMLER BAŞARIYLA TAMAMLANDI!")
    print("="*80)
    
    return pipeline, results

# Çalıştır
pipeline_final, results_final = run_complete_pipeline()

## 10. Özet ve Kontroller

In [ ]:
print("="*80)
print("PIPELINE ÖZET RAPORU")
print("="*80)

print("\n1. KONTROL LİSTESİ:")
print("-"*40)

checks = [
    ("Tek pipeline kullanıldı", True),
    ("Scoring default kapalı", config.enable_scoring == False),
    ("Data dictionary kullanıldı", data_dictionary is not None),
    ("Two-stage calibration yapıldı", 'calibration_stage1' in results),
    ("Forward/Backward/Stepwise mevcut", config.selection_method in ['forward', 'backward', 'stepwise']),
    ("Boruta LightGBM ile çalıştı", 'boruta' in config.selection_order),
    ("WOE hesaplandı", 'woe_results' in results),
    ("Univariate Gini hesaplandı", 'univariate_gini' in results.get('woe_results', {})),
    ("IV optimizasyonu yapıldı", config.binning_method == 'optimized'),
    ("Selection sırası doğru", config.selection_order == ['psi', 'vif', 'correlation', 'iv', 'boruta', 'stepwise']),
    ("Tüm modeller denendi", config.model_type == 'all'),
    ("Noise sentinel kontrolü", config.enable_noise_sentinel),
    ("Risk bands hesaplandı", 'risk_bands' in results),
    ("Equal default splits", config.equal_default_splits),
    ("En az 5 değişken seçildi", len(results.get('selected_features', [])) >= 5)
]

for check_name, check_result in checks:
    status = "✓" if check_result else "✗"
    print(f"{status} {check_name}")

print("\n2. PERFORMANS ÖZETİ:")
print("-"*40)

if 'scores' in results and 'best_model_name' in results:
    best_scores = results['scores'].get(results['best_model_name'], {})
    
    print(f"En İyi Model: {results['best_model_name']}")
    
    if 'train_auc' in best_scores:
        train_gini = (best_scores['train_auc'] * 2 - 1) * 100
        print(f"  Train Gini: {train_gini:.1f}%")
    
    if 'test_auc' in best_scores:
        test_gini = (best_scores['test_auc'] * 2 - 1) * 100
        print(f"  Test Gini: {test_gini:.1f}%")
        
        if 60 <= test_gini <= 70:
            print(f"  ✓ Hedef aralıkta (60-70%)")
        else:
            print(f"  ✗ Hedef dışında")
    
    if 'oot_auc' in best_scores:
        oot_gini = (best_scores['oot_auc'] * 2 - 1) * 100
        print(f"  OOT Gini: {oot_gini:.1f}%")

print("\n" + "="*80)
print("✓ TÜM KONTROLLER TAMAMLANDI")
print("="*80)